In [2]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('Data')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

2024-11-12 17:51:04.551714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 17:51:04.728697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 17:51:04.775951: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 17:51:05.124453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-12 17:51:06.662704: W tensorflow/compiler/tf2

['fake', 'real']
Types of classes labels found:  2


In [4]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('Data' + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])

In [6]:
# Build a dataframe        
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

  Labels                            image
0   fake  dataset_path/fake/fake_1338.jpg
1   fake   dataset_path/fake/fake_568.jpg
2   fake   dataset_path/fake/fake_410.jpg
3   fake  dataset_path/fake/fake_1376.jpg
4   fake   dataset_path/fake/fake_303.jpg
     Labels                            image
2995   real  dataset_path/real/real_1088.jpg
2996   real  dataset_path/real/real_1496.jpg
2997   real   dataset_path/real/real_166.jpg
2998   real  dataset_path/real/real_1327.jpg
2999   real  dataset_path/real/real_1416.jpg


In [8]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  3000
Labels
fake    1500
real    1500
Name: count, dtype: int64


In [10]:
import cv2
path = 'Data/'
dataset_path = os.listdir('Data')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [12]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(3000, 224, 224, 3)

In [14]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['fake' 'fake' 'fake' ... 'real' 'real' 'real']
[0 0 0 ... 1 1 1]


In [16]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [18]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=42)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(2850, 224, 224, 3)
(2850, 2)
(150, 224, 224, 3)
(150, 2)


# EfficientNet Implementation :

In [21]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

I0000 00:00:1731466308.635749   32193 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731466308.997880   32193 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731466309.000949   32193 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731466309.006953   32193 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [23]:
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

model.summary()
with tf.device('/GPU:0'):
    hist = model.fit(train_x, train_y, epochs=50, verbose=2)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 2)              │     4,052,133 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,052,133 (15.46 MB)

 Trainable params: 4,010,110 (15.30 MB)

 Non-trainable params: 42,023 (164.16 KB)

2024-11-12 17:51:56.004710: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1716019200 exceeds 10% of free system memory.
2024-11-12 17:51:57.651894: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1716019200 exceeds 10% of free system memory.


Epoch 1/50


I0000 00:00:1731466338.546555   32341 service.cc:146] XLA service 0x7e392c002a60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731466338.546576   32341 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-11-12 17:52:19.257524: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-12 17:52:21.763331: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
E0000 00:00:1731466357.013290   32341 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1731466357.129412   32341 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsigh

90/90 - 88s - 979ms/step - accuracy: 0.5021 - loss: 2.5589
Epoch 2/50
90/90 - 8s - 91ms/step - accuracy: 0.5088 - loss: 1.4482
Epoch 3/50
90/90 - 8s - 91ms/step - accuracy: 0.5256 - loss: 1.0691
Epoch 4/50
90/90 - 8s - 91ms/step - accuracy: 0.4979 - loss: 1.2877
Epoch 5/50
90/90 - 8s - 91ms/step - accuracy: 0.5060 - loss: 1.0296
Epoch 6/50
90/90 - 8s - 91ms/step - accuracy: 0.5032 - loss: 0.9123
Epoch 7/50
90/90 - 8s - 91ms/step - accuracy: 0.5144 - loss: 0.9228
Epoch 8/50
90/90 - 8s - 91ms/step - accuracy: 0.5154 - loss: 0.8591
Epoch 9/50
90/90 - 8s - 91ms/step - accuracy: 0.5182 - loss: 0.8843
Epoch 10/50
90/90 - 8s - 91ms/step - accuracy: 0.5821 - loss: 0.8380
Epoch 11/50
90/90 - 8s - 91ms/step - accuracy: 0.6449 - loss: 0.8260
Epoch 12/50
90/90 - 8s - 92ms/step - accuracy: 0.7046 - loss: 0.6843
Epoch 13/50
90/90 - 8s - 92ms/step - accuracy: 0.7488 - loss: 0.6066
Epoch 14/50
90/90 - 8s - 92ms/step - accuracy: 0.7523 - loss: 0.6001
Epoch 15/50
90/90 - 8s - 92ms/step - accuracy: 0.764

In [25]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

2024-11-12 18:00:23.664045: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2056', 20 bytes spill stores, 20 bytes spill loads

2024-11-12 18:00:23.747787: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2056', 248 bytes spill stores, 248 bytes spill loads



3/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8247 - loss: 0.6465

2024-11-12 18:00:26.944181: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2056', 392 bytes spill stores, 400 bytes spill loads



5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7936 - loss: 0.7822
Loss = 0.9358620047569275
Test Accuracy = 0.753333330154419


In [27]:
model.save('real_fake_classifier_model1.keras')

# Testing Efficient Model On Unseen data

In [31]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the model
model = tf.keras.models.load_model('real_fake_classifier_model1.keras')

# Function to preprocess and predict
def classify_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Corrected size to match model input
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5:
        print("Fake")
    else:
        print("Real")

# Test with a new image
classify_image('real_0.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Real
